# loading data

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import  tensorflow as tf
from    tensorflow import  keras
from    tensorflow.keras import datasets, layers, optimizers, models
from    tensorflow.keras import regularizers
import  os

import  argparse
import  numpy as np


import matplotlib.pyplot as plt

import IPython.display as display

from PIL import Image

from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
import pathlib


class LoadFishDataUtil():
    def __init__(self, directory_str,BATCH_SIZE,IMG_SIZE):
      self.directory_str=directory_str
      self.BATCH_SIZE=BATCH_SIZE
      self.IMG_SIZE=IMG_SIZE
      self.data_dir = pathlib.Path(directory_str)
      self.image_count = len(list(self.data_dir.glob('*/*.png')))
      self.CLASS_NAMES = np.array([item.name for item in self.data_dir.glob('*') if item.name != "LICENSE.txt"])
      self.class_num=len(self.CLASS_NAMES)
 
      IMG_HEIGHT = IMG_SIZE
      IMG_WIDTH = IMG_SIZE
      self.STEPS_PER_EPOCH = np.ceil(self.image_count/BATCH_SIZE)


    def get_label(self,file_path):
    # convert the path to a list of path components
      parts = tf.strings.split(file_path, '/')
    # The second to last is the class-directory
      print(parts[-2] == self.CLASS_NAMES)
      wh = tf.where(tf.equal(self.CLASS_NAMES,parts[-2]))
      return parts[-2] == self.CLASS_NAMES
    def decode_img(self,img):
    # convert the compressed string to a 3D uint8 tensor
      img = tf.image.decode_jpeg(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
      img = tf.image.convert_image_dtype(img, tf.float32)
    #img = (img/127.5) - 1
    # resize the image to the desired size.
      return tf.image.resize(img, [self.IMG_SIZE, self.IMG_SIZE])

    def process_path(self,file_path):
      label = self.get_label(file_path)
    # load the raw data from the file as a string
      img = tf.io.read_file(file_path)
      img = self.decode_img(img)
      return img, label
 

    def prepare_for_training(self,ds, cache=True, shuffle_buffer_size=1000):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
      if cache:
        if isinstance(cache, str):
          ds = ds.cache(cache)
        else:
          ds = ds.cache()

      ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat forever
      ds = ds.repeat()

      ds = ds.batch(self.BATCH_SIZE)

  # `prefetch` lets the dataset fetch batches in the background while the model
  # is training.
      ds = ds.prefetch(buffer_size=self.AUTOTUNE)

      return ds
  

    def loadFishData(self):
      list_ds = tf.data.Dataset.list_files(str(self.data_dir/'*/*'))
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
      self.AUTOTUNE = tf.data.experimental.AUTOTUNE
      self.labeled_ds = list_ds.map(self.process_path, num_parallel_calls=self.AUTOTUNE)
   
      train_size = int(0.7 * self.image_count)
      val_size = int(0.15 * self.image_count)
      test_size = int(0.15 * self.image_count)
      train_ds = self.prepare_for_training(self.labeled_ds)

      full_dataset = train_ds.shuffle(buffer_size=1000,reshuffle_each_iteration = False )
      train_dataset = full_dataset.take(train_size)
      test_dataset = full_dataset.skip(train_size)
      val_dataset = test_dataset.skip(val_size)
      test_dataset = test_dataset.take(test_size)
      return train_dataset,val_dataset,test_dataset,self.STEPS_PER_EPOCH,self.CLASS_NAMES,self.class_num

In [ ]:
data_dir ='/home/xingbo/Desktop/fish_identification/data/SESSION_AQUARIUM/SESSION1'
BATCH_SIZE = 8
IMG_SIZE=160

In [ ]:
myloadData = LoadFishDataUtil(data_dir,BATCH_SIZE,IMG_SIZE)
train_dataset,val_dataset,test_dataset,STEPS_PER_EPOCH, CLASS_NAMES,class_num = myloadData.loadFishData()



# build model

In [ ]:
class VGG16(models.Model):


    def __init__(self, input_shape,num_classes):
        """

        :param input_shape: [32, 32, 3]
        """
        super(VGG16, self).__init__()

        weight_decay = 0.000
        self.num_classes = num_classes

        model = models.Sequential()

        model.add(layers.Conv2D(64, (3, 3), padding='same',
                         input_shape=input_shape, kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.3))

        model.add(layers.Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

        model.add(layers.Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

        model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))


        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))


        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Dropout(0.5))

        model.add(layers.Flatten())
        model.add(layers.Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(self.num_classes))
        # model.add(layers.Activation('softmax'))


        self.model = model


    def call(self, x):

        x = self.model(x)

        return x

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16


In [ ]:
base_model=VGG16(input_shape=[IMG_SIZE, IMG_SIZE, 3],include_top=False, weights='imagenet', classes=class_num)
for image_batch, label_batch in train_dataset.take(1):
   pass

image_batch.shape
feature_batch = base_model(image_batch)
print(feature_batch.shape)

## Feature extraction
You will freeze the convolutional base created from the previous step and use that as a feature extractor, add a classifier on top of it and train the top-level classifier.

### Freeze the convolutional base
It's important to freeze the convolutional based before you compile and train the model. By freezing (or setting `layer.trainable = False`), you prevent the weights in a given layer from being updated during training. MobileNet V2 has many layers, so setting the entire model's trainable flag to `False` will freeze all the layers.

In [ ]:
base_model.trainable = False
# Let's take a look at the base model architecture
base_model.summary()

### Add a classification head

Apply a `tf.keras.layers.Dense` layer to convert these features into a single prediction per image. You don't need an activation function here because this prediction will be treated as a `logit`, or a raw prediction value.  Positive numbers predict class 1, negative numbers predict class 0.

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

prediction_layer = keras.layers.Dense(class_num, activation=K.relu)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [ ]:
model = tf.keras.Sequential([
  base_model,
    global_average_layer,
  prediction_layer
])
model.summary()

In [ ]:
# must specify from_logits=True!
criteon = keras.losses.CategoricalCrossentropy(from_logits=True)
metric = keras.metrics.CategoricalAccuracy()

base_learning_rate = 0.01
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=criteon,
              metrics=[metric])

## let's train it

In [ ]:
initial_epochs = 100
validation_steps = 20

history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=val_dataset,validation_steps=validation_steps)

In [ ]:




#model = VGG16([IMG_SIZE, IMG_SIZE, 3], class_num)

# must specify from_logits=True!
criteon = keras.losses.CategoricalCrossentropy(from_logits=True)
metric = keras.metrics.CategoricalAccuracy()

optimizer = optimizers.Adam(learning_rate=0.01)

for epoch in range(250):

    for step, (x, y) in enumerate(train_dataset):
        # [b, 1] => [b]
        y = tf.squeeze(y, axis=1)
        # [b, 10]
        y = tf.one_hot(y, depth=class_num)

        with tf.GradientTape() as tape:
            logits = model(x)
            loss = criteon(y, logits)
            # loss2 = compute_loss(logits, tf.argmax(y, axis=1))
            # mse_loss = tf.reduce_sum(tf.square(y-logits))
            # print(y.shape, logits.shape)
            metric.update_state(y, logits)

        grads = tape.gradient(loss, model.trainable_variables)
        # MUST clip gradient here or it will disconverge!
        grads = [tf.clip_by_norm(g, 15) for g in grads]
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if step % 40 == 0:
            # for g in grads:
            #     print(tf.norm(g).numpy())
            print(epoch, step, 'loss:', float(loss), 'acc:', metric.result().numpy())
            metric.reset_states()

    if epoch % 1 == 0:

        metric = keras.metrics.CategoricalAccuracy()
        for x, y in val_dataset:
            # [b, 1] => [b]
            y = tf.squeeze(y, axis=1)
            # [b, 10]
            y = tf.one_hot(y, depth=class_num)

            logits = model.predict(x)
            # be careful, these functions can accept y as [b] without warnning.
            metric.update_state(y, logits)
        print('test acc:', metric.result().numpy())
        metric.reset_states()



